In [59]:
import cPickle as pickle
import sys
sys.path.append('/usr/app/frustum-pointnets/train/')
import provider
import numpy as np

In [60]:
data_path = '/usr/app/frustum-pointnets/train/detection_results_pyun_debug2.pickle'

In [61]:
with open(data_path, 'rb') as fp:
    ps_list = pickle.load(fp)
    segp_list = pickle.load(fp)
    center_list = pickle.load(fp)
    heading_cls_list = pickle.load(fp)
    heading_res_list = pickle.load(fp)
    size_cls_list = pickle.load(fp)
    size_res_list = pickle.load(fp)
    rot_angle_list = pickle.load(fp)
    score_list = pickle.load(fp)
    onehot_list = pickle.load(fp)
    obj_xyz_list = pickle.load(fp)


In [64]:
idx = 0
ps = ps_list[idx]
segp = segp_list[idx]
obj_xyz = obj_xyz_list[idx]

segp_mask = segp == 1
ps_mask = ps[segp_mask]
obj_xyz_mask = ps[segp_mask]
print("obj_xyz range {} - {}".format(obj_xyz[segp_mask].min(), obj_xyz[segp_mask].max()))
h,w,l,tx,ty,tz,ry = provider.from_prediction_to_label_format(center_list[idx],
    heading_cls_list[idx], heading_res_list[idx],
    size_cls_list[idx], size_res_list[idx], rot_angle_list[idx])
print(h,w,l,tx,ty,tz,ry)
# obj_xyz_rcv = obj_xyz * 2.0 - 1 
# obj_xyz_rcv = obj_xyz_rcv * np.array([l/2.0, h/2.0, w/2.0])
# obj_xyz_rcv = obj_xyz_rcv - np.array([tx, ty, tz])
# obj_xyz_rcv = provider.rotate_pc_along_y(obj_xyz_rcv, - ry)


obj_xyz range 0.126897796988 - 0.675606846809
(1.7785790423896504, 0.54572582437650685, 1.6330315032403182, 4.5964602326873667, 1.0669273962749339, 46.105614565488224, -2.09533688234416)


In [68]:
print("center point is at{}".format(np.array([tx, ty, tz])))
print("box3d size is {}".format(np.array([l, h, w])))
print("ps shape is {}".format(ps.shape))
print("ps range is {} - {}".format(ps[:,:3].min(axis=0), ps[:, :3].max(axis=0)))

center point is at[  4.59646023   1.0669274   46.10561457]
box3d size is [ 1.6330315   1.77857904  0.54572582]
ps shape is (1024, 4)
ps range is [ -0.27145693  -0.31816778  35.26903152] - [  0.55270344   1.28290808  61.69417572]
